In [73]:
import os
import csv
import time
import json
import psutil
import requests
import argparse
import subprocess
import gpcconstants

In [3]:
def download_single_apk(apk_id, apk_folder, index, total):
    # remove existing APK
    if os.path.exists(apk_folder + '/' + apk_id + '.apk'): os.remove(apk_folder + '/' + apk_id + '.apk')

    # download APK
    print("(" + index + "/" + total + ")" + "Downloading: " + apk_id)
    try:
        subprocess.check_output(['gplaycli','-d',apk_id,'-f',apk_folder])
    except subprocess.CalledProcessError as e:
        print("**** Failed to download due to error: ", e)
        return False
    
    if not os.path.exists(apk_folder + '/' + apk_id + ".apk"):
        print("**** Error occurred while downloading...")
        return False
    else: return True

In [30]:
def start_api_server(gscraper_dir):
    try:
        p = subprocess.Popen('npm start', cwd=gscraper_dir,shell=True,preexec_fn=os.setsid)
    except subprocess.CalledProcessError as e:
        print("Failed to download due to error: ", e)

In [28]:
def stop_api_server():
    for proc in psutil.process_iter():
        # check whether the process name matches
        if proc.name() == 'node':
            proc.kill()

In [66]:
def get_app_by_collection_category(collection, category, num, start):
    # get the list of top apps
    if category=='NONE':
        request_params = {'collection':collection, 'num':num, 'start':start, 'country':'ca'}
    else:
        request_params = {'collection':collection, 'category':category, 'num':num, 'start':start,'country':'ca'}    
    r = requests.get('http://localhost:3000/api/apps/', params=request_params)
    
    # if we successfully get the response
    if(r.ok):
        app_list = r.json().get('results')
        app_id_list = [app.get('appId') for app in app_list]
        return app_id_list
    else:
        # If response code is not ok (200), print the resulting http error code with description
        r.raise_for_status()

In [78]:
def download_batch_app(apk_folder, collection, category, num, start):
    # start the server as we need to query information of each app
    start_api_server(gpcconstants.GSCRAPPER_DIR)
    time.sleep(1)
    
    app_list = get_app_by_collection_category(collection, category, num, start)
    
    # variables to store app information which will be written to a csv file after we are done with one category
    app_info = {}
    
    # loop variables
    num_apps = len(app_list)
    index = 1
    
    # for each app in the app list: download the app and store the app information
    for app in app_list:
        if not download_single_apk(app, apk_folder, num_apps, index):
            app_info[app]=["Download Error"]
        else:
            complete_app_info=requests.get('http://localhost:3000/api/apps/'+app).json()
            app_info[app] = [complete_app_info.get('title'),complete_app_info.get('genreId'),complete_app_info.get('size'),complete_app_info.get('version'),complete_app_info.get('installs'),complete_app_info.get('scoreText'),complete_app_info.get('ratings'),complete_app_info.get('androidVersionText')]
        index = index + 1
    
    # stop the server
    stop_api_server()
    
    # write the collected app information to a csv file
    with open(apk_folder+'/app_info.csv','w+') as reportFile:
        wr = csv.writer(reportFile, dialect='excel')
        for app in app_info.keys():
            wr.writerow([app] + app_info.get(app))

In [76]:
if __name__ == '__main__':
    # construct the argument parser
    ap = argparse.ArgumentParser(description='Download batch apks from Google Play Store.')
    ap.add_argument('-d','--dir', required=True, help='The folder to store the downloaded APKs (required)')
    ap.add_argument('-a','--all', nargs='?', const=True, default=False,type=bool, help='Set this option to True will download all categories (default: False)')
    ap.add_argument('-c','--collection', nargs='?', default='TOP_FREE', help='The collection to download from {TOP_FREE,NEW_FREE,GROSSING,TRENDING} (default: TOP_FREE)')
    ap.add_argument('-g','--category', nargs='?', default='NONE', help='The category to download (default: NONE; Available options can be found in documentation).')
    ap.add_argument('-n','--number', nargs='?', default='120', help='The number of apps to download (default: 120; Max: 120).')
    ap.add_argument('-s','--start', nargs='?', default='0', help='The starting index of the retrieved list (default: 0; Max: 500)')
    
    # parse the arguments
    args = vars(ap.parse_args())
    apk_folder = args['dir']
    download_all = args['all']
    collection = gpcconstants.COLLECTION.get(args['collection'])
    category = gpcconstants.CATEGORY.get(args['category'])
    number = args['number']
    start = args['start']
    
    # get the app list
    if download_all:
        print("Downloading all possible categories")
        for category in gpcconstants.CATEGORY:
            apk_folder_new = os.path.join(apk_folder, category)
            if not os.path.exists(apk_folder_new):
                os.makedirs(apk_folder_new)
            download_batch_app(apk_folder_new, collection, category, number, start)
    else:
        download_batch_app(apk_folder, collection, category, number, start)

usage: ipykernel_launcher.py [-h] -d DIR [-a [ALL]] [-c [COLLECTION]]
                             [-g [CATEGORY]] [-n [NUMBER]] [-s [START]]
ipykernel_launcher.py: error: the following arguments are required: -d/--dir


SystemExit: 2

/Users/dulinglai/Applications/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [79]:
stop_api_server()

In [23]:
apk_folder = '/Volumes/WD_drive/TopFreeApps_201807'
app_id = 'io.voodoo.holeio'
download_single_apk(app_id,apk_folder)

Downloading: io.voodoo
An unknown error occurred while downloading...


False

In [8]:
GSCRAPPER_DIR = os.path.abspath('./google-play-api-master')
pid = start_api_server(GSCRAPPER_DIR)

In [62]:
get_app_by_collection_category()

['io.voodoo.holeio', 'air.com.nbcuni.com.telemundo.envivo', 'com.riseup.game', 'com.h8games.helixjump', 'com.playgendary.kickthebuddy', 'com.bigframes.color_road', 'com.facebook.orca', 'com.google.android.apps.youtube.music', 'com.dvloper.granny', 'com.snapchat.android', 'com.instagram.android', 'com.pandora.android', 'com.dimico.painthit', 'com.contextlogic.wish', 'com.rubygames.slingdrift', 'com.netflix.mediaclient', 'com.supertapx.lovedots', 'com.whatsapp', 'com.zhiliaoapp.musically', 'com.facebook.katana', 'com.roblox.client', 'com.playgendary.tanks', 'com.spotify.music', 'com.CraftingExplore.Big', 'com.supertapx.drawin', 'com.foxsports.android', 'com.bitstrips.imoji', 'com.kiloo.subwaysurf', 'com.word.puzzle.game.connect', 'com.google.android.projection.gearhead', 'com.amazon.mShop.android.shopping', 'com.google.android.play.games', 'com.ubercab', 'com.zplay.willhero', 'io.voodoo.cube', 'net.peakgames.toonblast', 'air.com.hypah.io.slither', 'com.crazylabs.lady.bug', 'com.abtnproje

In [77]:
app = 'io.voodoo.holeio'
apk_folder = '/Volumes/WD_drive/TopFreeApps_201807'

app_info_list = {}
complete_app_info=requests.get('http://localhost:3000/api/apps/'+app).json()
app_info_list[app] = [complete_app_info.get('title'),complete_app_info.get('genreId'),complete_app_info.get('size'),complete_app_info.get('version'),complete_app_info.get('installs'),complete_app_info.get('scoreText'),complete_app_info.get('ratings'),complete_app_info.get('androidVersionText')]
print(app_info_list[app])
output_file = apk_folder+'/app_info.csv'
with open(output_file,'w+') as reportFile:
    wr = csv.writer(reportFile, dialect='excel')
    for app in app_info_list.keys():
        wr.writerow([app] + app_info_list.get(app))

['Hole.io', 'GAME_ARCADE', '73M', '1.1.2', '1,000,000+', '4.1', 14556, '4.1 and up']
